# Testing
### Eheschliessung nach Herkunft der Eheschliessenden, seit 1993
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [6]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [7]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [8]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_ehe_herkunft_od3301

(int_dwh)

In [10]:
# Filepath
fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_ehe_herkunft_OD3301\BEV330OD3301.csv"


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [11]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [12]:
data2betested.head(3)

,StichtagDatJahr,HerkunftKomb,HerkunftKombCd,AnzSchl
0,1993,CHmitCH,1,1143
1,1993,CHmitAus,2,767
2,1993,AusmitCH,3,615


In [13]:
data2betested.dtypes

StichtagDatJahr     int64
HerkunftKomb       object
HerkunftKombCd      int64
AnzSchl             int64
dtype: object

In [14]:
data2betested.shape

(112, 4)

Beschreibe einzelne Attribute

In [15]:
data2betested.describe()

,StichtagDatJahr,HerkunftKombCd,AnzSchl
count,112,112,112
mean,2006,2,850
std,8,1,225
min,1993,1,427
25%,2000,2,652
50%,2006,2,851
75%,2013,3,1034
max,2020,4,1363


Wie viele Nullwerte gibt es im Datensatz?

In [16]:
data2betested.isnull().sum()

StichtagDatJahr    0
HerkunftKomb       0
HerkunftKombCd     0
AnzSchl            0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [17]:
data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,StichtagDatJahr,HerkunftKomb,HerkunftKombCd,AnzSchl
0,1993-01-01,CHmitCH,1,1143
1,1993-01-01,CHmitAus,2,767
2,1993-01-01,AusmitCH,3,615
3,1993-01-01,AusmitAus,4,1046
4,1994-01-01,CHmitCH,1,1121
...,...,...,...,...
107,2019-01-01,AusmitAus,4,1200
108,2020-01-01,CHmitCH,1,923
109,2020-01-01,CHmitAus,2,427
110,2020-01-01,AusmitCH,3,505


Bereite die Daten für die Grafik unten vor

In [18]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

28

In [19]:
for key, group in Jahre:
    print(key, len(group))

1993-01-01 00:00:00 4
1994-01-01 00:00:00 4
1995-01-01 00:00:00 4
1996-01-01 00:00:00 4
1997-01-01 00:00:00 4
1998-01-01 00:00:00 4
1999-01-01 00:00:00 4
2000-01-01 00:00:00 4
2001-01-01 00:00:00 4
2002-01-01 00:00:00 4
2003-01-01 00:00:00 4
2004-01-01 00:00:00 4
2005-01-01 00:00:00 4
2006-01-01 00:00:00 4
2007-01-01 00:00:00 4
2008-01-01 00:00:00 4
2009-01-01 00:00:00 4
2010-01-01 00:00:00 4
2011-01-01 00:00:00 4
2012-01-01 00:00:00 4
2013-01-01 00:00:00 4
2014-01-01 00:00:00 4
2015-01-01 00:00:00 4
2016-01-01 00:00:00 4
2017-01-01 00:00:00 4
2018-01-01 00:00:00 4
2019-01-01 00:00:00 4
2020-01-01 00:00:00 4


In [20]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

,StichtagDatJahr,HerkunftKomb,HerkunftKombCd,AnzSchl
108,2020-01-01,CHmitCH,1,923
109,2020-01-01,CHmitAus,2,427
110,2020-01-01,AusmitCH,3,505
111,2020-01-01,AusmitAus,4,896


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [23]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [24]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [25]:
data2betested

,StichtagDatJahr,HerkunftKomb,HerkunftKombCd,AnzSchl
0,1993-01-01,CHmitCH,1,1143
1,1993-01-01,CHmitAus,2,767
2,1993-01-01,AusmitCH,3,615
3,1993-01-01,AusmitAus,4,1046
4,1994-01-01,CHmitCH,1,1121
...,...,...,...,...
107,2019-01-01,AusmitAus,4,1200
108,2020-01-01,CHmitCH,1,923
109,2020-01-01,CHmitAus,2,427
110,2020-01-01,AusmitCH,3,505


In [27]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]
#mySelection[[mySelection.NationFHistLang =='Schweiz']]


highlight = alt.selection(type='single', on='mouseover',
                          fields=['HerkunftKomb'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(data2betested).encode(
    x='StichtagDatJahr',
    y='AnzSchl',
    color='HerkunftKomb'
)

points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [31]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='AnzSchl',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('HerkunftKomb', scale=alt.Scale(scheme='dark2')),
    tooltip=['AnzSchl','StichtagDatJahr','HerkunftKomb','HerkunftKombCd']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**